[Extracting NHS Data Medium](https://medium.com/towards-data-science/nhs-english-prescribing-data-analysis-using-python-9d86ec610880)
- [github for article](https://github.com/Adenrele/Hayfever-NHS-Prescribing-Data-)


[NHS Hack Day - Datasets and APIs](https://nhshackday.com/resources/datasets-and-apis)

Using the [ckan](https://docs.ckan.org/en/2.8/api/) API to extract data from the NHS website. The data is then cleaned and analysed.

## Data
[English Prescribing Datase (EPD)](https://opendata.nhsbsa.net/dataset/english-prescribing-data-epd)


[English Prescribing Dataset (EPD) with SNOMED Code](https://opendata.nhsbsa.net/dataset/english-prescribing-dataset-epd-with-snomed-code)

In [1]:
import requests
import pandas as pd


In [2]:
#The url, described within the API documentation, is used to view the datasets available.
url = "http://opendata.nhsbsa.net/api/3/action/package_list"
#Request to obtain data from a url.
response = requests.get(url)
#Status code 200 means the request was successful.
response.status_code
#Display the response as a dictionary.
response.json()

#Print the values in the ‘result’ key.
#The ‘result’ key contains the names of datasets which are available.
print(response.json()["result"])

['23362', '25521', 'consolidated-pharmaceutical-list', 'english-prescribing-data-epd', 'english-prescribing-dataset-epd-with-snomed-code', 'foi-01003', 'foi-01004', 'foi-01005', 'foi-01008', 'foi-01016', 'foi-01017', 'foi-01035', 'foi-18062', 'foi-18076', 'foi-19119', 'foi-19158', 'foi-19162', 'foi-19167', 'foi-19173', 'foi-19174', 'foi-19177', 'foi-19181', 'foi-19192', 'foi-19193', 'foi-19200', 'foi-19205', 'foi-19208', 'foi-19210', 'foi-19214', 'foi-20213', 'foi-20215', 'foi-20225', 'foi-20227', 'foi-20235', 'foi-20237', 'foi-20241', 'foi-20242', 'foi-20243', 'foi-20245', 'foi-20246', 'foi-20247', 'foi-20248', 'foi-20249', 'foi-20251', 'foi-20253', 'foi-20256', 'foi-20257', 'foi-20264', 'foi-21266', 'foi-21267', 'foi-21268', 'foi-21270', 'foi-21271', 'foi-21273', 'foi-21276', 'foi-21278', 'foi-21279', 'foi-21280', 'foi-22276', 'foi-22277', 'foi-22278', 'foi-22280', 'foi-22281', 'foi-22283', 'foi-22290', 'foi-22291', 'foi-22295', 'foi-22296', 'foi-22298', 'foi-22300', 'foi-22302', 'fo

In [3]:
# We could use the original epd or the experimental snomed version
epd = 'english-prescribing-data-epd'
epd_sno = 'english-prescribing-dataset-epd-with-snomed-code'

In [4]:
#Using the API documentation, the url containing the medicines data #was deduced and tested.
url2 = "http://opendata.nhsbsa.net/api/3/action/package_show?id=english-prescribing-data-epd"
response2 = requests.get(url2)
response2_json = response2.json() 
response2.status_code
#The keys within the output of the results were investigated.
response2_json['result'].keys()

dict_keys(['num_resources', 'maintainer', 'tags', 'relationships_as_object', 'private', 'maintainer_email', 'num_tags', 'groups', 'license_id', 'isopen', 'id', 'metadata_created', 'organization', 'name', 'metadata_modified', 'author', 'author_email', 'notes', 'license_title', 'owner_org', 'relationships_as_subject', 'url', 'state', 'version', 'license_url', 'title', 'creator_user_id', 'revision_id', 'type', 'resources', 'extras'])

In [5]:
# Each dataset for medicines in secondary care is stored within csv files downloadable through a url.
response2_json["result"]["resources"][0]["url"]

'https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/8ae6b792-2a0c-4f4b-826c-dc6483dc32a7/download/epd_201401.csv'

In [6]:
# Each url is passed into a list.
datasets =[]
for num in range(len(response2_json['result']['resources'])):
    datasets.append(response2_json['result']['resources'][num]['url'])
datasets

['https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/8ae6b792-2a0c-4f4b-826c-dc6483dc32a7/download/epd_201401.csv',
 'https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/78b8c360-1048-4d30-82fd-02e66fa8b693/download/epd_201402.csv',
 'https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/54584067-3109-4f27-8c48-c78001923c3b/download/epd_201403.csv',
 'https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/5e25a419-8334-4fb2-bcef-8508870be90e/download/epd_201404.csv',
 'https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/5763be28-0dde-430c-bddc-73b32a575d75/download/epd_201405.csv',
 'https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/32c2c600-2e9d-47e2-9d45-407d0e6b03f7/download/epd_201406.csv',
 'https://opendata.nhsbsa.net/dataset/65050ec0-5abd-48ce-989d-defc08ed837e/resource/9f185ff8-aad4-4555-b1fe-ed49560b6a

In [19]:
import pandas as pd
import numpy as np
test_df = pd.read_csv(datasets[0], nrows=1, sep =",", chunksize=100,  error_bad_lines=False)

C:\Users\dec2g\AppData\Local\Temp\ipykernel_10220\2550546182.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test_df = pd.read_csv(datasets[0], nrows=1, sep =",", chunksize=100,  error_bad_lines=False)


OverflowError: Python int too large to convert to C long

In [21]:
import csv
import sys
csv.field_size_limit(2147483)

csv.reader(datasets[0], delimiter=',', quoting=csv.QUOTE_NONE)

test_df = pd.read_csv(datasets[0], nrows=1, sep =",", chunksize=100,  error_bad_lines=False)

C:\Users\dec2g\AppData\Local\Temp\ipykernel_10220\2704217197.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test_df = pd.read_csv(datasets[0], nrows=1, sep =",", chunksize=100,  error_bad_lines=False)


OverflowError: Python int too large to convert to C long

In [10]:
# https://stackoverflow.com/questions/25962114/how-do-i-read-a-large-csv-file-with-pandas/25962187#25962187

import dask.dataframe as dd
df = dd.read_csv(datasets[0], sep =",")
df.to_csv('epd.csv')



ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------+---------+----------+
| Column    | Found   | Expected |
+-----------+---------+----------+
| ADDRESS_2 | object  | float64  |
| ADQUSAGE  | float64 | int64    |
+-----------+---------+----------+

The following columns also raised exceptions on conversion:

- ADDRESS_2
  ValueError("could not convert string to float: 'MARGARET STREET'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'ADDRESS_2': 'object',
       'ADQUSAGE': 'float64'}

to the call to `read_csv`/`read_table`.

## Reading the Data
"Attempting to read every CSV in the list above results in wastage of hours of waiting followed by a Jupyter Notebooks crash. To begin, I investigated the memory usage of one dataframe.
After exploring the data further to determine the most suitable features, records and data types, the function below was created. It saves memory and time by; reading only the required columns, changing the data types to those that use less memory where appropriate and by filtering the records as required."

In [ ]:
from datetime import datetime
def LargeCSVsChop(listofCSV_URLs):
    
    
    dfs = []
    for num in range(len(listofCSV_URLs)):
        feats = ['YEAR_MONTH', 'PCO_NAME', 'PRACTICE_NAME', 'CHEMICAL_SUBSTANCE_BNF_DESCR', 
            'BNF_DESCRIPTION', 'TOTAL_QUANTITY', 'ACTUAL_COST']
            
        custom_date_parser = lambda x: datetime.strptime(x, "%Y%m")
        
        df = pd.read_csv(listofCSV_URLs[num], sep =",", usecols = feats, date_parser=custom_date_parser,
                                                                            parse_dates=['YEAR_MONTH'])
        
        """"
        'feats' is a list of the columns/features from the CSV that I want Pandas to read to memory.
        
        types = {'TOTAL_QUANTITY': int, 'ACTUAL_COST':float}
        
        Types is a dictionary of columns and the datatypes they should be read as using dtypes = types in the read_csv function. 
        
        The types are optimal so no changes will be made but the dictionary can be used in future.    
        
        
        'custom_data_parser' will be used to convert the dtype of the YEAR_MONTH feature to date format.
        The dataframe will read only feats and convert the dtype of the YEAR_MONTH feature.
        
        'df' is used as a variable again for the filtered dataframe to prompt the deallocation of the unfiltered
        #dataframe object from memory by dropping the reference count to zero and prompting garbage collection algorithms.
        
        
        Oral antihistamines will be the focus of this analysis. 'CHEMICAL_SUBSTANCE_BNF_DESCR' is used to ensure all
        forms and brands of the drugs are captured. Further work is done using 'BNF_DESCRIPTION' to remove unwanted products.
        
        """
        
        df = df[ 
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Cetirizine hydrochloride')                              | 
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Loratadine')                                            |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Desloratadine')                                         |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Fexofenadine hydrochloride')                            |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Acrivastine')                                           |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Bilastine')                                             |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Levocetirizine')                                        |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Mizolastine')                                           |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Chlorphenamine maleate')                                & 
                (df['BNF_DESCRIPTION']!='Chlorphenamine 10mg/1ml solution for injection ampoules')            |
                (df['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Promethazine hydrochloride')                            &
                (df['BNF_DESCRIPTION']!='Promethazine 25mg/1ml solution for injection ampoules')              &
                (df['BNF_DESCRIPTION']!='Phenergan 25mg/1ml solution for injection ampoules')                   
                ]
        
        dfs.append(df)
        
    df_large = pd.concat(dfs)
        
    return df_large
Jan_2019_df = LargeCSVsChop(datasets[60:61])
Jan_2019_df.info(memory_usage='deep', null_counts=True, verbose=False)

OverflowError: Python int too large to convert to C long

In [ ]:
def LargeCSVsChop(listofCSV_URLs):
    feats = ['YEAR_MONTH', 'PCO_NAME', 'PRACTICE_NAME', 'CHEMICAL_SUBSTANCE_BNF_DESCR', 
             'BNF_DESCRIPTION', 'TOTAL_QUANTITY', 'ACTUAL_COST']
    custom_date_parser = lambda x: datetime.strptime(x, "%Y%m")
    dfs = []
    for url in listofCSV_URLs:
        for chunk in pd.read_csv(url, usecols=feats, parse_dates=['YEAR_MONTH'], 
                                 date_parser=custom_date_parser, chunksize=100000):
            chunk = chunk[(chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Cetirizine hydrochloride') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Loratadine') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Chlorphenamine maleate') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Promethazine hydrochloride') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Diphenhydramine hydrochloride') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Mirtazapine') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Amitriptyline hydrochloride') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Fluoxetine hydrochloride') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Citalopram hydrobromide') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Sertraline hydrochloride') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Escitalopram oxalate') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Venlafaxine') | 
                          (chunk['CHEMICAL_SUBSTANCE_BNF_DESCR']=='Duloxetine hydrochloride') | 
                          (chunk['BNF_DESCRIPTION']!='Phenergan 25mg/1ml solution for injection ampoules')]
            dfs.append(chunk)
    return pd.concat(dfs, ignore_index=True)

In [ ]:
Jan_2019_df = LargeCSVsChop(datasets[60:61])
Jan_2019_df.info(memory_usage='deep', null_counts=True, verbose=False)

OverflowError: Python int too large to convert to C long

In [ ]:
import time
import numpy as np

def timefunc(function, arg, repeats = 20):
    
    alltime = []
    
    while  repeats > 0:
        
        """
        Unless specified, the number of repitions will be 20.
        
        
        """
        
        starttime= time.time() # record the start time

        result = function(arg) # run the function and store in the variable 'result' in case result is needed.

        endtime = time.time() # Record end time.

        timetaken = endtime - starttime 
        
        alltime.append(timetaken) 
        
        repeats -=1  
        
    mean = np.mean(alltime) #Find the mean.
    std = np.std(alltime) #Find the standard deviation.
    error=std/(len(alltime)**0.5)   #Find the standard error.
    
    return (mean)

In [ ]:
def FuncReadCSV(listofdataframes):
    
    frames = []
    for csv in listofdataframes:
        df = pd.read_csv(csv)
        frames.append(df)
        dfs= pd.concat(frames)
    return df

Without_ncols = timefunc(FuncReadCSV, datasets[60:62], repeats = 1)

#Return the mean timetaken to run read a list of CSVs without removing columns or filtering records.

OverflowError: Python int too large to convert to C long

In [ ]:
def LargeCSVsChop_MINI(listofCSV_URLs):
    
    """
    The same as LargeCSVsChop without the filtering element. This is to enable like for like
    comparisons as much as possible (with the exception of date parsing) as it pertains to time

    """
    dfs = []
    for num in range(len(listofCSV_URLs)):

        feats = ['YEAR_MONTH', 'PCO_NAME', 'PRACTICE_NAME', 'CHEMICAL_SUBSTANCE_BNF_DESCR', 
                'BNF_DESCRIPTION', 'TOTAL_QUANTITY', 'ACTUAL_COST']
        
        custom_date_parser = lambda x: datetime.strptime(x, "%Y%m")
        
        df = pd.read_csv(listofCSV_URLs[num], sep =",", usecols = feats, date_parser=custom_date_parser,
                                                                         parse_dates=['YEAR_MONTH'])
      
        dfs.append(df)
        
    df_large = pd.concat(dfs)
        
    return df_large



With_ncols = timefunc(LargeCSVsChop_MINI, datasets[60:62], repeats = 1)

#Return the mean time taken to read one CSV using the LargeCSVsChop_MINI function.